In [51]:
import numpy as np 
import pandas as pd 
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio #Plotly theming
pio.templates.default = 'plotly_white'
pio.templates["plotly_white"] = go.layout.Template(
    layout=go.Layout(
        colorway=px.colors.qualitative.Set2,
        title_font_color="green"
    )
)


In [52]:
df = pd.read_csv('world_population.csv')
df.drop(columns=["Rank", "CCA3", "Capital","World Population Percentage"], inplace=True)
fig = px.treemap(df, path=["Continent", "Country"], values="2022 Population")
fig.show()

1.We can see that the Asian continent contribute to 50% of world population globally.
2.The top two contries contribute to population in Asia are china and India.

In [54]:
years = []
total_pops = []
col_list = df.columns
for col in range(2,10):
    sum = df[col_list[col]].sum()
    years.append(col_list[col].replace(" Population",""))
    total_pops.append(sum)
df_year = pd.DataFrame({"Year" : years, "Population" : total_pops}).sort_values(by="Year")
df_year["Year"]=pd.to_datetime(df_year["Year"])
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_year["Year"], y=df_year["Population"]))
fig.update_layout(title="World Population by Years")
fig.show()

1.On seeing the above graph we can assume that the population is increasing in average of one billion per decade.

In [55]:
fig = go.Figure(data=go.Choropleth(
    locations=df['Country'],
    z = df['Growth Rate'],
    locationmode = 'country names', 
    colorscale = 'jet',
    colorbar_title = "Growth Rate",
))

fig.update_layout(
    title = dict(text = '<b>Countries Population Growth Rate</b>',
    x = 0.5)
)

fig.show()

The 5 countries with the highest growth rates are Moldova, Polandia, Niger, Syria, and Slovakia by 2022.
If it continues, Moldova's population will reach 5 times more by 2050.

In [39]:
colors = ["#1d7874","#679289","#f4c095","#ee2e31","#ffb563","#918450","#f85e00","#a41623","#9a031e","#d6d6d6","#ffee32","#ffd100","#333533","#202020"]
by_continent = df.groupby('Continent').sum().sort_values(by = '2022 Population',ascending = True)
cont_gr = by_continent[['Growth Rate']].sort_values(by = 'Growth Rate',ascending = True)
cont_gr
data = go.Bar(x = cont_gr.index, y = cont_gr['Growth Rate'], text = cont_gr['Growth Rate'],textposition ='outside',
              textfont = dict(size = 12,
                             color = 'black'),
              marker = dict(color = colors,
                            opacity = 0.7,
                            line_color = 'black',
                            line_width = 2))
layout = go.Layout(title = {'text': "<b>Continents Wise Population Growth Rate</b>",
                           'x':0.5,
                           'xanchor': 'center'},
                   xaxis = dict(title='<b>Continents</b>'),
                   yaxis =dict(title='<b>Growth Rate</b>'),
                   width = 900,
                   height = 600,
                   template = 'plotly_white')
fig=go.Figure(data = data, layout = layout)

fig.show()

As we have observed growth rate by countries as a map before , we can conclude that African countries will contribute to world population the most in future

In [40]:
def cal_population_projection(df, start_year, target_year):
    start_year_pop = str(start_year) + " Population"
    target_year_pop = str(target_year) + " Population"
    start_year_gr = str(start_year) + " Growth Rate"
    df[target_year_pop] = df[start_year_pop] * ((1 + (df[start_year_gr]/100)) ** (target_year - start_year))
    df[target_year_pop] = df[target_year_pop].astype(int)
    return df

def cal_gr_estimation(df, start_year, target_year):
    start_year_pop = str(start_year) + " Population"
    target_year_pop = str(target_year) + " Population"
    target_year_gr = str(target_year) + " Growth Rate"
    df[target_year_gr] = ((df[target_year_pop]/df[start_year_pop]) ** (1/(target_year-start_year)) - 1) * 100
    return df

def cal_density(df, year):
    den_year = str(year) + " Density (per km²)"
    pop_year = str(year) + " Population"
    df[den_year] = df[pop_year] / df["Area (km²)"]
    return df

def cal_pop_percentage(df, year):
    pop_percentage_year = str(year) + " World Population Percentage"
    pop_year = str(year) + " Population"
    df[pop_percentage_year] = (df[pop_year] / df[pop_year].sum()) * 100
    return df


df_population = df
# Population Column
pop_col = [x for x in df_population.columns.tolist() if x[5:]=="Population"]
clean_pop_col = [int(x.replace(" Population", "")) for x in pop_col]
country_pop_col = [x for x in df_population.columns.tolist() if x=="Country" or x[5:]=="Population"]

# Growth Rate Column
gr_col = [x for x in df_population.columns.tolist() if x[5:]=="Growth Rate"]
clean_gr_col = [int(x.replace(" Growth Rate", "")) for x in gr_col]
country_gr_col = [x for x in df_population.columns.tolist() if x=="Country" or x[5:]=="Growth Rate"]
# Rename and Change Current Columns
df_population = df_population.rename(columns={"Growth Rate": "2022 Growth Rate"})
df_population["1970 Growth Rate"] = "Unknown"
df_population["2022 Growth Rate"] = (df_population["2022 Growth Rate"] - 1) * 100
df_population = df_population.rename(columns={"Density (per km²)": "2022 Density (per km²)"})
df_population = df_population.rename(columns={"World Population Percentage": "2022 World Population Percentage"})

# Calculate Population Projection
target_year = [2030, 2040, 2050]
for year in target_year:
    df_population = cal_population_projection(df_population, 2022, year)

# Calculate Growth Rate
start_year = [1970, 1980, 1990, 2000, 2010, 2015, 2022, 2022, 2022]
target_year = [1980, 1990, 2000, 2010, 2015, 2020, 2030, 2040, 2050]
for i in range(len(target_year)):
    df_population = cal_gr_estimation(df_population, start_year[i], target_year[i])

target_year = [1970, 1980, 1990, 2000, 2010, 2015, 2020, 2030, 2040, 2050]
# Calculate Density
for i in range(len(target_year)):
    df_population = cal_density(df_population, target_year[i])

# Calculate World Population Percentage
for i in range(len(target_year)):
    df_population = cal_pop_percentage(df_population, target_year[i])

# Population Column
pop_col = [x for x in df_population.columns.tolist() if x[5:]=="Population"]
clean_pop_col = [int(x.replace(" Population", "")) for x in pop_col]
country_pop_col = [x for x in df_population.columns.tolist() if x=="Country" or x[5:]=="Population"]

# Growth Rate Column
gr_col = [x for x in df_population.columns.tolist() if x[5:]=="Growth Rate"]
clean_gr_col = [int(x.replace(" Growth Rate", "")) for x in gr_col]
country_gr_col = [x for x in df_population.columns.tolist() if x=="Country" or x[5:]=="Growth Rate"]

total_population = df_population[pop_col].sum(axis=0)
total_population.index = clean_pop_col
total_population = total_population.sort_index()

display(total_population)

x = [1970, 2022,2040]
y = [total_population[v] for v in x]

fig = px.area(total_population, color_discrete_sequence=['#3455EB'])

fig.update_traces(patch={"line": {"width": 3}})

fig.add_trace(
    go.Scatter(x=x,
               y=y,
               mode='markers',
               marker=dict(
                 color="#0926AD",
                 size=12,
              )
    )
)

fig.add_vrect(x0=2023, x1=2055,
              fillcolor="#dbd9d9", opacity=0.5,
              layer="below", line_width=0,
)

fig.add_annotation(yref='paper',
                   x=1994, y=-0.15,
                   text="<b>Actual<b>",
                   showarrow=False,
)

fig.add_annotation(yref='paper',
                   x=2037, y=-0.15,
                   text="<b>Forecast<b>",
                   showarrow=False,
)

fig.update_xaxes(title_text="", 
                 range=[1968,2052], 
                 showgrid=False, 
                 linecolor='black',
                 ticks='outside',
)

fig.update_yaxes(title_text="", 
                 range=[3000000000,11000000000], 
                 showgrid=False, 
                 linecolor='black'
)

fig.update_layout(title_text='World Population Growth, 1970-2050',
                  width=1000, 
                  height=450,
                  plot_bgcolor='white',
                  showlegend=False,
)

fig.show()

In [ ]:
From year to year, world population is increasing.
World population is projected to reach 10 billion people by 2050.